<a href="https://www.kaggle.com/code/chandlertimm/black-clover-manga-colorization?scriptVersionId=95692241" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Import Libraries

In [1]:
import os
import sys
import random
import math
import re
import time
import numpy as np
import json
import cv2
import matplotlib
import matplotlib.pyplot as plt
import scipy
import warnings
import shutil
import random
import shutil 

from PIL import Image
from tqdm import tqdm

# Process the dataset

In [2]:
!cp -r ../input/black-clover-manga-dataset/colored ./
!cp -r ../input/black-clover-manga-dataset/grayscale ./

In [3]:
# colored
colored_files =  os.listdir('./colored')
sorted_colored =  sorted(colored_files, key=lambda x: int(os.path.splitext(x)[0]))

# grayscale
grayscale_files =  os.listdir('./grayscale')
sorted_grayscale =  sorted(grayscale_files, key=lambda x: int(os.path.splitext(x)[0]))

In [4]:
# shuffle the dataset
temp = list(zip(sorted_colored, sorted_grayscale))

random.shuffle(temp)
res1, res2 = zip(*temp)

# res1 and res2 come out as tuples, and so must be converted to lists.
colored, grayscale = list(res1), list(res2)

print(colored[0:5])
print(grayscale[0:5])

['223.png', '90.png', '305.png', '600.png', '399.png']
['223.png', '90.png', '305.png', '600.png', '399.png']


In [5]:
#split the dataset 0.8 for training

#training colored
train_cl = []
for i in tqdm(range(int(0.0*len(colored)), int(0.8*len(colored)))):
    train_cl.append(colored[i])
    
#testing grayscale
test_cl = []
for i in tqdm(range(int(0.8*len(colored)), int(1.0*len(colored)))):
    test_cl.append(colored[i])

100%|██████████| 126/126 [00:00<00:00, 405588.87it/s]


In [6]:
print(train_cl[-3:None])
print(test_cl[0:3])

['491.png', '462.png', '512.png']
['154.png', '30.png', '415.png']


In [7]:
#training grayscale
train_bw = []
for i in tqdm(range(int(0.0*len(grayscale)), int(0.8*len(grayscale)))):
    train_bw.append(grayscale[i])
    
#testing grayscale
test_bw = []
for i in tqdm(range(int(0.8*len(grayscale)), int(1.0*len(grayscale)))):
    test_bw.append(grayscale[i])

100%|██████████| 126/126 [00:00<00:00, 204521.02it/s]


In [8]:
print(train_bw[-3:None])
print(test_bw[0:3])

['491.png', '462.png', '512.png']
['154.png', '30.png', '415.png']


In [9]:
# create the dataset folders
if not os.path.isdir('./blackclover'):
    os.makedirs('./blackclover')
    # unpaired image dataset path
    os.makedirs('./blackclover/trainA/')
    os.makedirs('./blackclover/trainB/')
    os.makedirs('./blackclover/testA/')
    os.makedirs('./blackclover/testB/')

In [10]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

In [11]:
# copy the train dataset to unpaired image dataset path

for i in tqdm(range(len(train_cl)), 'copying train colored and grayscale'):
    # Source path 
    src1 = find(str(train_bw[i]), './grayscale/')
    src2 = find(str(train_cl[i]), './colored/')

    # Destination path 
    dest1 = './blackclover/trainA/'
    dest2 = './blackclover/trainB/'

    # Copy the content of source to destination 
    shutil.copy(src1, dest1) 
    shutil.copy(src2, dest2)  

copying train colored and grayscale: 100%|██████████| 504/504 [00:01<00:00, 283.67it/s]


In [12]:
# copy the test dataset to unpaired image dataset path

for i in tqdm(range(len(test_cl)), 'copying test colored and grayscale'):
    # Source path 
    src1 = find(str(test_bw[i]), './grayscale/')
    src2 = find(str(test_cl[i]), './colored/')

    # Destination path 
    dest1 = './blackclover/testA/'
    dest2 = './blackclover/testB/'

    # Copy the content of source to destination 
    shutil.copy(src1, dest1) 
    shutil.copy(src2, dest2) 

copying test colored and grayscale: 100%|██████████| 126/126 [00:00<00:00, 183.38it/s]


# Install MMGeneration